# Tested on ml.p4de.24xlarge
PyTorch 2.0.0, Python 3.10, CUDA Version 11.8

In [2]:
!nvidia-smi

Mon Jan  1 22:13:38 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.8     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:10:1C.0 Off |                    0 |
| N/A   22C    P0    59W / 400W |      0MiB / 81251MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM...  Off  | 00000000:10:1D.0 Off |                    0 |
| N/A   

In [4]:
# Matches the CUDA version reported by nvidia-smi (11.8, in this case)
# Configure here: https://pytorch.org/
%pip install -U torch --index-url https://download.pytorch.org/whl/cu118
%pip install -U transformers==4.36.2
%pip install -U peft==0.7.1
%pip install -U datasets==2.15.0
%pip install -U bitsandbytes==0.41.2
%pip install -U scipy==1.10.1
%pip install -U ipywidgets==8.1.1
%pip install -U matplotlib==3.7.4

Looking in indexes: https://download.pytorch.org/whl/cu118, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 GB 412.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.1.2
    Uninstalling torch-2.1.2:
      Successfully uninstalled torch-2.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.12 requires torch<2.1,>=1.7, but you have torch 2.1.2+cu118 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com

[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.ne

In [6]:
!python -m bitsandbytes

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
++++++++++++++++++ BUG REPORT INFORMATION ++++++++++++++++++
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

++++++++++++++++++ /usr/local CUDA PATHS +++++++++++++++++++
/usr/local/cuda-11.8/compat/libcuda.so

+++++++++++++++ WORKING DIRECTORY CUDA PATHS +++++++++++++++


++++++++++++++++++ LD_LIBRARY CUDA PATHS +++++++++++++++++++
++++++++++++++++ /opt/conda/lib CUDA PATHS +++++++++++++++++
/opt/conda/lib/libicudata.so
/opt/conda/lib/python3.10/site-packages/torch/lib/libc10_cuda.so
/opt/conda/lib/python3.10/site-packages/torch/lib/libtorch_cuda.so
/opt/conda/lib/python3.10/site-packages/torch/lib/libtorch_cuda_linalg.so
/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110.so
/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda110_nocublaslt.so
/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda111.so
/opt/conda/lib/python3.10/site-packages/bits

In [2]:
from accelerate import FullyShardedDataParallelPlugin, Accelerator
from torch.distributed.fsdp.fully_sharded_data_parallel import FullOptimStateDictConfig, FullStateDictConfig

fsdp_plugin = FullyShardedDataParallelPlugin(
    state_dict_config=FullStateDictConfig(offload_to_cpu=True, rank0_only=False),
    optim_state_dict_config=FullOptimStateDictConfig(offload_to_cpu=True, rank0_only=False),
)

accelerator = Accelerator(fsdp_plugin=fsdp_plugin)

Detected kernel version 4.14.330, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [3]:
from datasets import load_dataset

dataset_name = "databricks/databricks-dolly-15k"

train_dataset = load_dataset(dataset_name, split="train[0:800]")
eval_dataset = load_dataset(dataset_name, split="train[800:1000]")

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

base_model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, 
                                             quantization_config=bnb_config, 
                                             device_map="auto")

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

## Setup instruction dataset

Must follow this format:  https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1

In [41]:
# Tokenization 
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left",
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token

In [42]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = f"""

[INST] Given a question and some additional context, provide an answer. [/INST]

### Question:
{data_point['instruction']}

### Context:
{f"Here is some context: {data_point['context']}" if len(data_point["context"]) > 0 else ""}

### Response:
{data_point['response']}

</s>
"""

    tokenized_prompt = tokenizer(full_prompt)
    return tokenized_prompt

tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

untokenized_text = tokenizer.decode(tokenized_train_dataset[0]['input_ids']) 
print(untokenized_text)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

<s> 

[INST] Given a question and some additional context, provide an answer. [/INST]

### Question:
When did Virgin Australia start operating?

### Context:
Here is some context: Virgin Australia, the trading name of Virgin Australia Airlines Pty Ltd, is an Australian-based airline. It is the largest airline by fleet size to use the Virgin brand. It commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route. It suddenly found itself as a major airline in Australia's domestic market after the collapse of Ansett Australia in September 2001. The airline has since grown to directly serve 32 cities in Australia, from hubs in Brisbane, Melbourne and Sydney.

### Response:
Virgin Australia commenced services on 31 August 2000 as Virgin Blue, with two aircraft on a single route.
</s> 



In [19]:
# #max_length = 400 

# tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
# tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

# untokenized_text = tokenizer.decode(tokenized_train_dataset[4]['input_ids']) 
# print(untokenized_text)

In [20]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [21]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "w1",
        "w2",
        "w3",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

# Apply the accelerator. You can comment this out to remove the accelerator.
model = accelerator.prepare_model(model)

trainable params: 120350720 || all params: 23602952192 || trainable%: 0.5098968934945001


In [22]:
!rm -rf ./dolly_mixtral_finetune

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [23]:
import transformers
from datetime import datetime

if torch.cuda.device_count() > 1: # If more than 1 GPU
    model.is_parallelizable = True
    model.model_parallel = True

output_dir = "./dolly_mixtral_finetune"

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=1,
        gradient_checkpointing=True,
        gradient_accumulation_steps=4,
        max_steps=10,
        learning_rate=2.5e-5, 
        logging_steps=5,
        fp16=True, 
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=10,               # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=10,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Detected kernel version 4.14.330, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
10,1.738700,1.662252


/opt/conda/lib/python3.10/site-packages/peft/utils/save_and_load.py:131: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


TrainOutput(global_step=10, training_loss=1.8725128173828125, metrics={'train_runtime': 171.1996, 'train_samples_per_second': 0.234, 'train_steps_per_second': 0.058, 'total_flos': 2634833590456320.0, 'train_loss': 1.8725128173828125, 'epoch': 0.05})

In [39]:
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    padding_side="left"
)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [46]:
prompt = """
<s>
[INST] Given a question and some additional context, provide an answer. [/INST]

### Question:
Who wrote "Generative AI on AWS" by O'Reilly Media?

### Context: 
Chris Fregly and Antje Barth wrote "Data Science on AWS" by O'Reilly Media.
Chris Fregly, Antje Barth, and Shelbee Eigenbrode wrote "Generative AI on AWS" by O'Reilly Media.

### Response:</s>
"""

model_input = eval_tokenizer(prompt, return_tensors="pt").to("cuda")

with torch.no_grad():
    print(eval_tokenizer.decode(model.generate(**model_input, max_new_tokens=32)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



 
[INST] Given a question and some additional context, provide an answer. [/INST]

### Question:
Who wrote "Generative AI on AWS" by O'Reilly Media?

### Context: 
Chris Fregly and Antje Barth wrote "Data Science on AWS" by O'Reilly Media.
Chris Fregly, Antje Barth, and Shelbee Eigenbrode wrote "Generative AI on AWS" by O'Reilly Media.

### Response:
 
Chris Fregly, Antje Barth, and Shelbee Eigenbrode wrote "Generative AI on AWS" by O'Reilly Media


# Load PEFT model and perform inference

In [24]:
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# base_model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_id, 
#     quantization_config=bnb_config,
#     device_map="auto"
# )

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/92.7k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
# from peft import PeftModel

# peft_model = PeftModel.from_pretrained(base_model, f"{output_dir}/checkpoint-10")
# #peft_model.eval()

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained(
#     base_model_id,
# #    add_bos_token=True
# )
# tokenizer.pad_token = tokenizer.eos_token

# eval_tokenizer = AutoTokenizer.from_pretrained(
#     base_model_id,
#     add_bos_token=True,
# )

In [ ]:
# prompt = """

# [INST] Given a question and some additional context, provide an answer. [/INST]

# ### Question:
# Who wrote "Generative AI on AWS" by O'Reilly Media?

# ### Context: 
# Chris Fregly and Antje Barth wrote "Data Science on AWS" by O'Reilly Media.
# Chris Fregly, Antje Barth, and Shelbee Eigenbrode wrote "Generative AI on AWS" by O'Reilly Media.

# ### Response:

# """

# model_input = tokenizer(prompt, return_tensors="pt").to("cuda")

# with torch.no_grad():
#     print(tokenizer.decode(peft_model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))